In [ ]:
from gssutils import *

if is_interactive():
    scraper = Scraper("https://www.gov.uk/government/statistics/regional-trade-in-goods-statistics-dis-aggregated-by-smaller-geographical-areas-2017")
    tabs = {tab.name: tab for tab in scraper.distribution(title=lambda t: 'Data Tables' in t).as_databaker()}

In [ ]:
tab = tabs['T4 NUTS2 Partner Country']

In [ ]:
tidy = pd.DataFrame()

In [ ]:
flow = tab.filter('Flow').fill(DOWN).is_not_blank().is_not_whitespace()
geography = tab.filter('Partner Country').fill(DOWN).is_not_blank().is_not_whitespace() 
nut = tab.filter('NUTS2').fill(DOWN).is_not_blank().is_not_whitespace() 
observations = tab.filter('Statistical Value (£ million)').fill(DOWN).is_not_blank().is_not_whitespace()
observations = observations.filter(lambda x: type(x.value) != str or 'HMRC' not in x.value)
Dimensions = [
            HDim(flow,'Flow',DIRECTLY,LEFT),
            HDim(geography,'Geography',DIRECTLY,LEFT),
            HDim(nut,'NUTS Geography',DIRECTLY,LEFT),
            HDimConst('SITC 4', 'all'),
            HDimConst('Measure Type', 'GBP Total'),
            HDimConst('Unit', 'gbp-million'),
            HDimConst('Year', dataset_year)
            ]
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
table1 = c1.topandas()
tidy = pd.concat([tidy, table1])

In [ ]:
savepreviewhtml(c1)

In [ ]:
observations1 = tab.filter('Business Count').fill(DOWN).is_not_blank().is_not_whitespace()
observations1 = observations1.filter(lambda x: type(x.value) != str or 'HMRC' not in x.value)
Dimensions = [
            HDim(flow,'Flow',DIRECTLY,LEFT),
            HDim(geography,'Geography',DIRECTLY,LEFT),
            HDim(nut,'NUTS Geography',DIRECTLY,LEFT),
            HDimConst('SITC 4', 'all'),
            HDimConst('Measure Type', 'Count of Businesses'),
            HDimConst('Unit', 'businesses'),
            HDimConst('Year', '2017')
            ]
c2 = ConversionSegment(observations1, Dimensions, processTIMEUNIT=True)
table2 = c2.topandas()
tidy = pd.concat([tidy, table2])

In [ ]:
savepreviewhtml(c2)

In [ ]:
import numpy as np
tidy['OBS'].replace('', np.nan, inplace=True)
tidy.dropna(subset=['OBS'], inplace=True)
tidy.drop(columns=['DATAMARKER'], inplace=True)
tidy.rename(columns={'OBS': 'Value'}, inplace=True)
tidy['Value'] = tidy['Value'].astype(int)
tidy['Value'] = tidy['Value'].map(lambda x:''
                                  if (x == ':') | (x == 'xx') | (x == '..') | (x == 'N/A')
                                  else int(x))

In [ ]:
for col in tidy.columns:
    if col not in ['Value', 'Year']:
        tidy[col] = tidy[col].astype('category')
        display(col)
        display(tidy[col].cat.categories)

In [ ]:
tidy['NUTS Geography'] = tidy['NUTS Geography'].cat.rename_categories({
    'All NUTS2 areas': 'nuts2/all',
    'Bedfordshire and Hertfordshire': 'nuts2/UKH2',
    'Berkshire, Buckinghamshire and Oxfordshire': 'nuts2/UKJ1',
    'Cheshire':'nuts2/UKD6',
    'Cornwall and Isles of Scilly':'nuts2/UKK3',
    'Cumbria':'nuts2/UKD1',
    'Derbyshire and Nottinghamshire':'nuts2/UKF1',
    'Devon':'nuts2/UKK4',
    'Dorset and Somerset':'nuts2/UKK2',
    'East Anglia':'nuts2/UKH1',
    'EA BTTA': 'nuts2/ea-btta',
    'EA Energy':'nuts2/ea-energy',
    'EA Other':'nuts2/ea-other',
    'East Wales':'nuts2/UKL2',
    'East Yorkshire and Northern Lincolnshire':'nuts2/UKE1',
    'Eastern Scotland':'nuts2/UKM7',
    'EM BTTA':'nuts2/em-btta',
    'EM Energy':'nuts2/em-energy',
    'EM Other':'nuts2/em-other',
    'Essex':'nuts2/UKH3',
    'Gloucestershire, Wiltshire and Bath/Bristol area':'nuts2/UKK1',
    'Greater Manchester':'nuts2/UKD3',
    'Hampshire and Isle of Wight':'nuts2/UKJ3',
    'Herefordshire, Worcestershire and Warwickshire':'nuts2/UKG1',
    'Highlands and Islands':'nuts2/UKM6',
    'Inner London - East':'nuts2/UKI4',
    'Inner London - West':'nuts2/UKI3',
    'Kent':'nuts2/UKJ4',
    'Lancashire':'nuts2/UKD4',
    'Leicestershire, Rutland and Northamptonshire':	'nuts2/UKF2',
    'Lincolnshire':'nuts2/UKF3',
    'LO BTTA':'nuts2/lo-btta',
    'LO Other':'nuts2/lo-other',
    'Merseyside':'nuts2/UKD7',
    'NE BTTA':'nuts2/ne-btta',
    'NE Energy':'nuts2/ne-energy',
    'NE Other':'nuts2/ne-other',
    'North Eastern Scotland':'nuts2/UKM5',
    'North Yorkshire':'nuts2/UKE2',
    'Northern Ireland':'nuts2/UKN0',
    'Northumberland and Tyne and Wear':	'nuts2/UKC2',
    'NW BTTA':'nuts2/nw-btta',
    'NW Energy':'nuts2/nw-energy',
    'NW Other':'nuts2/nw-other',
    'Outer London - East and North East':'nuts2/UKI5',
    'Outer London - South':'nuts2/UKI6',
    'Outer London - West and North West':'nuts2/UKI7',
    'SC BTTA':'nuts2/sc-btta',
    'SC Energy':'nuts2/sc-energy',
    'SC Other':'nuts2/sc-other',
    'SE BTTA':'nuts2/se-btta',
    'SE Energy':'nuts2/se-energy',
    'SE Other':'nuts2/se-other',
    'Shropshire and Staffordshire':	'nuts2/UKG2',
    'South Western Scotland':'nuts2/swsc',
    'SW BTTA':'nuts2/sw-btta',
    'SW Other':'nuts2/sw-other',
    'South Yorkshire':'nuts2/UKE3',
    'Southern Scotland':'nuts2/UKM9',
    'Surrey, East and West Sussex':'nuts2/UKJ2',
    'Tees Valley and Durham':'nuts2/UKC1',
    'West Central Scotland':'nuts2/UKM8',
    'West Midlands':'nuts2/UKG3',
    'West Wales':'nuts2/UKL1',
    'West Yorkshire':'nuts2/UKE4',
    'WA BTTA':'nuts2/wa-btta',
    'WA Energy':'nuts2/wa-energy',
    'WA Other':'nuts2/wa-other',
    'WM BTTA':'nuts2/wm-btta',
    'WM Other':'nuts2/wm-other',
    'YH BTTA':'nuts2/yh-btta',
    'YH Energy':'nuts2/yh-energy',
    'YH Other':'nuts2/yh-other'

})
tidy['Geography'] = tidy['Geography'].cat.rename_categories({
        'Congo (Republic)' : 'Congo (Democratic Republic of the)',
        'Dominican Rep' : 'Dominican Republic',
        'Equat Guinea' : 'Equatorial Guinea',
        'Falkland Islands' : 'Falklands Islands and dependencies',
        'Fyr Macedonia': 'Macedonia',
        'Irish Republic' : 'Republic of Ireland',
        'Russia' : 'Russian Federation',
        'Tanzania' : 'Tanzania (United Republic of)',
        'Trinidad:Tobago' : 'Trinidad and Tobago',
        'UAE' : 'United Arab Emirates',
        'USA' : 'United States',
        'Venezuela' : 'Venezuela, Bolivarian Republic of',
        'Residual Trade - no Partner Country displayed' : 'Residual Trade',
        'OtherLatin America and Caribbean' : 'Other Latin America and Caribbean',
        'Other Middle East and N Africa (excl EU)' : 'Other Middle East and North Africa'       
 
})
tidy['Flow'] = tidy['Flow'].cat.rename_categories({
        'Exp'   : 'exports',
        'Imp' : 'imports'})

In [ ]:
import urllib.request as request
import csv
import io
import requests

r = request.urlopen('https://raw.githubusercontent.com/ONS-OpenData/ref_trade/master/codelists/hmrc-geographies.csv').read().decode('utf8').split("\n")
reader = csv.reader(r)
url="https://raw.githubusercontent.com/ONS-OpenData/ref_trade/master/codelists/hmrc-geographies.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

tidy = pd.merge(tidy, c, how = 'left', left_on = 'Geography', right_on = 'Label')

tidy.columns = ['HMRC Partner Geography' if x=='Notation' else x for x in tidy.columns]

In [ ]:
tidy =tidy[['Year','NUTS Geography','HMRC Partner Geography','Flow','SITC 4','Measure Type', 'Value', 'Unit']]